<a href="https://colab.research.google.com/github/Otter2022/Oil_Marketing_Campaign/blob/main/tokens2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
#tokens2
import csv
import re
import sys
import os

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

source_data = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Source_Data/'
source_data_output = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Source_Data_Output/'
source_code = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Source_Code/'
admin = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Admin/'
backups = '/content/drive/MyDrive/ColabNotebooks/FinalProject_AI/Backups/'

#The lists contained in the marketing_compaign_oil.py file serves as a header for name pre-processing
#contains exclude, compiled_patterns for exclude, fix_abbreviations, honorifics, suffixes, suffixes_roman, etc.
marketing_campaign_oil = source_code+'marketing_campaign_oil.py'
with open(marketing_campaign_oil, 'r') as f:
    exec(f.read())
    f.close()

input_file = source_data+'tokens2_9071_CSV.csv'
#input_file = source_data + 'tokens2_test_csv.csv'
output_file = source_data_output+'tokens2_formatted_output.csv'


# Test for successul package import marketing_campaign_oil
print(honorifics)
print(suffixes)
print(suffixes_roman)


Mounted at /content/drive
['Dr', 'Doctor', 'Mr', 'Mrs', 'Ms', 'Miss', 'Prof', 'Professor', 'Sir', 'Madam', 'Dame', 'Lord', 'Lady', 'Rev', 'Reverend', 'Hon', 'Honorable', 'Capt', 'Captain', 'Major', 'Col', 'Colonel', 'Gen', 'General', 'Adm', 'Admiral', 'Sgt', 'Sergeant', 'Lt', 'Lieutenant', 'Chief', 'Officer', 'Judge', 'Magistrate', 'Sen', 'Senator', 'Rep', 'Representative', 'Amb', 'Ambassador', 'Pres', 'President', 'VP', 'Vice President', 'Gov', 'Governor', 'Mayor']
['Jr', 'Sr', 'Esq', 'Esquire', 'PhD', 'MD', 'DDS', 'DVM', 'JD', 'CPA', 'RN', 'LPN', 'PE', 'Ret']
['V', 'IV', 'III', 'II', 'I']


In [ ]:
def is_excluded(input_string):
    # Check if any compiled pattern matches the input string
    #I want to print the input_strings that are excluded
    for pattern in compiled_patterns:
        if pattern.match(input_string):
            print(f"Excluded: {input_string}")
            return True
    return False


def abbreviate(token):
  if token in fix_abbreviations.keys():
    return fix_abbreviations[token]
  else:
    return token


def rearrange_tokens(name):
    # Split the name into tokens
    tokens = name.split()

    # If there are exactly two tokens, rearrange them
    if len(tokens) == 2:
        token1, token2 = tokens[0].strip(), tokens[1].strip()

        #Fix abbreviations, i.e. FAM to Famil, TR to Trust
        token1 = abbreviate(token1)
        token2 = abbreviate(token2)

        # Capitalize the first letter of each token
        '''
        SMITH to Smith
        Except for suffixes like IV, V, III, II, I
        '''
        if token1 not in suffixes_roman:
          token1 = token1.capitalize()
        if token2 not in suffixes_roman:
          token2 = token2.capitalize()


        # EXCLUDE if any token for the owner name is in the exclude list
        for token in tokens:
          if is_excluded(token):
            print(f"Excluded: {name}")
            return "Miss"

        # 2-token sub-categories - General to specific
        if token1.isalpha() and token2.isalpha():
          if token1 in honorifics:
            return f"{token1} {token2}" #Dr. Smith, Mr. Smith
          elif token2 in suffixes:
            return f"{token1} {token2}" # Smith Jr
          elif token2 in suffixes_roman:
            return f"{token1} {token2}" #. Smith IV
          else:
            return f"{token2} {token1}" #John Smith
        elif token1.isalpha() and re.search(r'[#\d\-]', token2):
            return f"{token1} {token2}"#3 Trust #1234
    return "Miss"


def process_names(input_file, output_file):
    with open(input_file, 'r') as csvfile:
        reader = csv.reader(csvfile)
        names = [row[0] for row in reader]

    names = [name.strip() for name in names]

    formatted_names = [(name, rearrange_tokens(name)) for name in names]
    misses = 0
    total = len(formatted_names)
    print(f"Total: {total}")

    try:
        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['column1', 'column2'])  # Write headers
            for original, rearranged in formatted_names:
                if rearranged == 'Miss':
                    misses += 1
                else:
                    writer.writerow([original, rearranged])
    except TypeError:
        with open(output_file, 'w') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['column1', 'column2'])  # Write headers
            for original, rearranged in formatted_names:
                if rearranged == 'Miss':
                    misses += 1
                else:
                    writer.writerow([original, rearranged])

    return misses, total



In [ ]:
def main():
    misses,total = process_names(input_file, output_file)
    print(f"Total: {total}")
    print(f"Errors: {misses}")
    print(f"Miss percentage {(float(misses)/float(total))*100.00}%.")

if __name__ == "__main__":
    main()

Excluded: FUNDING
Excluded: HLF FUNDING
Excluded: ROYALTIES/HOLDINGS
Excluded: WAKEFIELD ROYALTIES/HOLDINGS
Excluded: FUNDERBURK
Excluded: FUNDERBURK JULIA
Excluded: UNKNOWN-LSE
Excluded: UNKNOWN-LSE #361068
Excluded: UNKNOWN-LSE
Excluded: UNKNOWN-LSE #361069
Excluded: UNKNOWN-LSE
Excluded: UNKNOWN-LSE #361070
Excluded: UNKNOWN-LSE
Excluded: UNKNOWN-LSE #361071
Excluded: UNKNOWN-LSE
Excluded: UNKNOWN-LSE #361072
Excluded: UNKNOWN-LSE
Excluded: UNKNOWN-LSE #361073
Excluded: UNKNOWN-LSE
Excluded: UNKNOWN-LSE 361136
Excluded: UNKNOWN-LSE
Excluded: UNKNOWN-LSE 361137
Excluded: UNKNOWN-LSE
Excluded: UNKNOWN-LSE 361139
Excluded: MANAGEMENT
Excluded: PEKF MANAGEMENT
Total: 9071
Total: 9071
Errors: 102
Miss percentage 1.1244625730349467%.
